### Insert Library

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
from datetime import datetime
import pandas as pd

### Pencarian list emiten menggunakan chrome webdriver

In [2]:
# Set up the WebDriver
driver = webdriver.Chrome()

# Go to IDX Stock List page
driver.get("https://www.idx.co.id/data-pasar/data-saham/daftar-saham/")

# Wait for table to load
try:
    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "vgt-table")))
except Exception as e:
    print(f"Timeout waiting for tables to load: {e}")

# Find and interact with the dropdown to select "all" items per page
try:
    # Find the select element using the correct class
    select_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, ".footer__row-count__select"))
    )
    
    # Use Select class to interact with the dropdown
    from selenium.webdriver.support.ui import Select
    select = Select(select_element)
    
    # Select "All" option which has value="-1"
    select.select_by_value("-1")
    
    # Wait longer for the table to refresh with all items
    time.sleep(5)
    
    print("Successfully selected 'All' from the dropdown")
except Exception as e:
    print(f"Error interacting with dropdown: {e}")
    driver.save_screenshot("dropdown_error.png")

# Now parse the page with all emitens displayed
print("Scraping all emitens...")
soup = BeautifulSoup(driver.page_source, "html.parser")
table = soup.find('table', {'id': 'vgt-table'})

emiten_list = []
if table:
    rows = table.find_all('tr')
    
    for row in rows[1:]:  # Skip header row
        cells = row.find_all('td')
        if cells and len(cells) > 0:
            ticker = cells[0].text.strip()
            if ticker:  # Make sure we got a valid ticker
                emiten_list.append(f"{ticker}.JK")
    
    print(f"Found {len(emiten_list)} emitens in total")
else:
    print("No table found on this page")

# Save with nice formatting
with open('emiten_list.json', 'w') as file:
    json.dump(emiten_list, file, indent=4)

print(f"Saved {len(emiten_list)} emitens to emiten_list.json")
driver.quit()

Successfully selected 'All' from the dropdown
Scraping all emitens...
Found 951 emitens in total
Saved 951 emitens to emiten_list.json
